### TITANIC DATA

- WE HAVE TITANIC DATA FROM KAGGLE , DECIMALIZED & READY. WE ARE PREDICTING WHICH PASSENGER WILL SURVIVE ON THE SINKING SHIP
- RUN DIFFERENT AI'S ON IT
- I HAVE ANSWERS TO THE TEST SET IN DB AND CSV FILES



In [62]:
from sklearn.datasets import *
from sqlalchemy import create_engine
import numpy
import pandas
from pandas import *
import pandas.io.sql as psql
import csv
import datetime
import matplotlib.pyplot as plt
from matplotlib import style
import matplotlib
import psycopg2
import glob, os
from collections import OrderedDict
import shutil
import pdb
from pdb import set_trace as bp
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier 
from sklearn.linear_model import LogisticRegression

In [63]:
fp=r'/Users/Aseem/titanic/' #path will change


In [64]:
# CONNECTING TO DB
conn=psycopg2.connect(database="AI_DB_LOCAL",user="Aseem",host="localhost")
cur = conn.cursor()

### RF
 RF CANNOT TELL YOU WHY IT CAME TO A PREDICTION

In [65]:
#RANDOM FOREST
df_train=pandas.read_csv(fp+'Train_1.csv')
df_test=pandas.read_csv(fp+'Test_1.csv')
features=df_train.columns[1:-1] #taking specific columns for training.ignoring ID and class column 

clf = RandomForestClassifier(n_estimators=350,n_jobs=2, random_state=0)# Create a random forest Classifier. By convention, clf means 'Classifier'

clf.fit(df_train[features], df_train['survived']) #Training Random Forest . clf.fit(data,target)
rf_pred=clf.predict(df_test[features])
rf_prob=clf.predict_proba(df_test[features])
decision_path
df_test_prob=pandas.DataFrame(np.column_stack([[rf_prob[i,0] for i in range(len(rf_prob))],[rf_prob[i,1] for i in range(len(rf_prob))],rf_pred,df_test['passenger_id']]),columns=['rf_prob_c0','rf_prob_c1','rf_pred','passenger_id'])

engine = create_engine('postgresql+psycopg2://Aseem:@localhost/AI_DB_LOCAL')

df_test_prob.to_sql('titanic_rf_test_prob', engine,schema='titanic',if_exists='replace',index=False)

rf_analysis=psql.read_sql("""
    select count(*) as total_trans,sum(accuracy) sum_of_accuracy,round((sum(accuracy)::numeric/count(*))*100,2) as total_accuracy_perc
    from (
        select *, case when rf_pred=survived::int then 1 else 0 end as accuracy
        from(
            select a.*,b.survived 
            from titanic.titanic_rf_test_prob a
            left join titanic.test_answers b
            on a.passenger_id=b.passengerid::int
            )a
        )b;"""
           ,conn  )
conn.commit()

print('Random Forest Accuracy\n',rf_analysis)

Random Forest Accuracy
    total_trans  sum_of_accuracy  total_accuracy_perc
0          418              306                73.21


In [70]:
importances = clf.feature_importances_
importances
clf.decision_path(df_train[features]) 

(<891x114734 sparse matrix of type '<class 'numpy.int64'>'
 	with 3291891 stored elements in Compressed Sparse Row format>,
 array([     0,    301,    638,    965,   1276,   1607,   1930,   2291,
          2630,   2983,   3342,   3693,   4030,   4337,   4652,   5027,
          5364,   5683,   6020,   6327,   6656,   6957,   7322,   7655,
          7990,   8333,   8624,   8975,   9304,   9615,   9920,  10261,
         10602,  10935,  11250,  11563,  11920,  12265,  12570,  12967,
         13286,  13637,  13962,  14293,  14640,  14993,  15292,  15605,
         15932,  16267,  16620,  17029,  17350,  17677,  17990,  18321,
         18662,  18999,  19314,  19621,  19960,  20307,  20660,  21023,
         21362,  21679,  21988,  22337,  22632,  22933,  23276,  23617,
         23960,  24243,  24576,  24923,  25286,  25645,  25968,  26287,
         26606,  26913,  27258,  27603,  27894,  28195,  28538,  28865,
         29184,  29511,  29886,  30239,  30598,  30945,  31308,  31643,
         319

### NIV2

In [53]:
#NIV2  conf_thresh=[0.8]  accuracy= [76.5%] 

df_test_prob_NI=pandas.read_csv(fp+'Probability/Test_1_prob.csv')
df_test_prob2_NI=pandas.DataFrame(np.column_stack([df_test_prob_NI['passenger_id'],df_test_prob_NI['NFM_C0'],df_test_prob_NI['NFM_C1']])
                 ,columns=['passenger_id','nfm_c0','nfm_c1'])
df_test_prob2_NI.to_sql('titanic_niv2_test_prob', engine,schema='titanic',if_exists='replace',index=False)

niv2_analysis=psql.read_sql("""
    --NIV2
    select count(*) as total_trans,sum(accuracy) sum_of_accuracy,round((sum(accuracy)::numeric/count(*))*100,2) as total_accuracy_perc
    from (
        select *,case when pred_class=survived::int then 1 else 0 end as accuracy
        from(
            select a.*,b.survived ,case when a.nfm_c1>a.nfm_c0 then 1 when a.nfm_c0>a.nfm_c1 then 0 else -1 end as pred_class
            from titanic.titanic_niv2_test_prob a
            left join titanic.test_answers b
            on a.passenger_id=b.passengerid::int
            )a
            where pred_class>-1
        )b
        ;"""
         ,conn  )
conn.commit()

print('NIV2 Accuracy\n',niv2_analysis)

NIV2 Accuracy
    total_trans  sum_of_accuracy  total_accuracy_perc
0          417              319                 76.5
